In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import nltk
from  nltk.sentiment import SentimentIntensityAnalyzer
from nltk import tokenize
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/youssefamine/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
PATH = 'cmu/'
movies = pd.read_csv(PATH + 'movie.metadata.tsv', sep='\t', header=None)
summaries = pd.read_csv(PATH + 'plot_summaries.txt', sep='\t', header=None)
summaries
summary_columns = ['Wikipedia Movie ID', 'Plot Summary']
summaries.columns = summary_columns
summaries

,Wikipedia Movie ID,Plot Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."


In [3]:
plot_summaries = summaries['Plot Summary']
summary = str(plot_summaries[0])
print(summary)

Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.


In [4]:
sid = SentimentIntensityAnalyzer()
print(summary)
ss = sid.polarity_scores(summary)
print(ss)
for k in sorted(ss):
   print('{0}: {1}, '.format(k, ss[k]), end='')


Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.
{'neg': 0.093, 'neu': 0.812, 'pos': 0.095, 'compound': 0.0083}
compound: 0.0083, neg: 0.093, neu: 0.812, pos: 0.095, 

In [6]:
data = {'Wikipedia Movie ID': [], 'compound': [], 'negative': [], 'neutral': [], 'postive': []}
for i in range(len(plot_summaries)):
    row = summaries.iloc[i]
    summary = str(row['Plot Summary'])
    ids = row['Wikipedia Movie ID']
    ss = sid.polarity_scores(summary)
    data['Wikipedia Movie ID'].append(ids)
    data['compound'].append(ss['compound'])
    data['negative'].append(ss['neg'])
    data['neutral'].append(ss['neu'])
    data['postive'].append(ss['pos'])

In [16]:
df = pd.DataFrame(data)
df

,Wikipedia Movie ID,compound,negative,neutral,postive
0,23890098,0.0083,0.093,0.812,0.095
1,31186339,-0.9946,0.137,0.786,0.078
2,20663735,-0.9923,0.189,0.697,0.114
3,2231378,-0.4226,0.112,0.778,0.109
4,595909,-0.9538,0.111,0.820,0.069
...,...,...,...,...,...
42298,34808485,0.2732,0.066,0.843,0.091
42299,1096473,0.9717,0.094,0.775,0.131
42300,35102018,0.9874,0.000,0.806,0.194
42301,8628195,0.8299,0.070,0.820,0.109


In [23]:
character_columns = ['Wikipedia Movie ID', 'Freebase Movie ID', 'Character Name', 'Actor DOB', 'Actor gender', 'Actor height', 'Actor ethnicity', 'Actor Name', 'Actor age at movie release', 'Freebase character map']
movies_columns = ['Wikipedia Movie ID', 'Freebase Movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres']
movies.columns = movies_columns
print(movies_columns)

['Wikipedia Movie ID', 'Freebase Movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres']


In [27]:
box_movies = movies[['Wikipedia Movie ID','Movie box office revenue']].dropna()
emotions = box_movies.join(df, on='Wikipedia Movie ID', how='inner')
emotions

ValueError: columns overlap but no suffix specified: Index(['Wikipedia Movie ID'], dtype='object')

In [25]:
box_movies

,Wikipedia Movie ID,Movie box office revenue
0,975900,14010832.0
7,10408933,3600000.0
13,171005,10161099.0
17,77856,102272727.0
21,612710,21521.0
...,...,...
81695,54540,288752301.0
81720,7761830,15369573.0
81725,1918494,6944471.0
81726,664006,22886222.0
